In [1]:
# import required modules

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import PoissonRegressor
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import statsmodels.api as sm

popsize = pd.read_csv("PopEstimates.csv")

In [2]:
## import data frames

Data31 = pd.read_csv("data/31DayDataModified.csv")

Data62 = pd.read_csv("data/62DayDataModified.csv")

popsize = pd.read_csv("PopEstimates.csv")


# define function

def find_popsize(HB,Q,popsize):
    year=Q[0:4]
    x=np.where(popsize['Health Board Code']==HB)
    popsize2=np.asarray(popsize[year])
    return popsize2[x]

In [3]:
# check
HB='S08000015'
Q='2012Q2'

find_popsize(HB,Q,popsize)

array([373220])

In [4]:
# edit 31 days data

df=Data31

# remove Nans
df = df[df['NumberOfEligibleReferrals31DayStandard'].notna()]
df = df[df['NumberOfEligibleReferralsTreatedWithin31Days'].notna()]

# remove 'all cancer types' from data
df = df[df['CancerType'] != 'All Cancer Types']

# add a numerical variable 1-1 to quarters and an index variable before/after pandemic
quarters=df['Quarter']
HBs=np.asarray(df['HB'])
date=np.zeros(len(quarters))
quars=df['Quarter'].unique()
dates=np.arange(len(quars))
pandemic=np.zeros(len(quarters))
pandemic_cat = np.zeros(len(quarters))
justpandemic=np.zeros(len(quarters))
pandemicRecovery=np.zeros(len(quarters))

x=np.where(quars=='2020Q1') # change to decide in which quarter the pandemic begins
z = np.where(quars == '2021Q1') # change to decide where recovery from pandemic begins.
   
pop_size=np.zeros(len(quarters))

for i in np.arange(len(quarters)):
    Q=quarters[i]
    HB=HBs[i]
    pop_size[i]=find_popsize(HB,Q,popsize)
    y=np.where(quars==quarters[i])
    date[i]=dates[y] # change if we don't want the nuemrical variable to be integers (what is best?)
    if dates[y] > x:
        justpandemic[i] = dates[y]-x
        pandemic_cat[i] = 1
    if dates[y] > x and dates[y] < z:
        pandemic[i] = dates[y]-x
    elif dates[y] > z:
        pandemicRecovery[i] = dates[y]-z
        


df['PopSize'] =pop_size.tolist()
df['Date'] =date.tolist()
df['Pandemic'] =pandemic.tolist()
df['Pandemic_cat'] =pandemic_cat.tolist()
df['JustPandemic'] =justpandemic.tolist()
df['PandemicRecovery'] =pandemicRecovery.tolist()
df.to_csv('data/31days.csv')

In [5]:
df

,Quarter,HB,CancerType,NumberOfEligibleReferrals31DayStandard,NumberOfEligibleReferralsTreatedWithin31Days,Date,Pandemic,PopSize,Pandemic_cat,JustPandemic,PandemicRecovery
0,2012Q1,S08000015,Breast,176,176,0.0,0.0,373220.0,0.0,0.0,0.0
1,2012Q1,S08000015,Cervical,10,10,0.0,0.0,373220.0,0.0,0.0,0.0
2,2012Q1,S08000015,Colorectal,112,110,0.0,0.0,373220.0,0.0,0.0,0.0
3,2012Q1,S08000015,Head & Neck,34,32,0.0,0.0,373220.0,0.0,0.0,0.0
4,2012Q1,S08000015,Lung,148,148,0.0,0.0,373220.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4994,2021Q3,S08000032,Lymphoma,56,56,38.0,0.0,661960.0,1.0,6.0,2.0
4995,2021Q3,S08000032,Melanoma,70,70,38.0,0.0,661960.0,1.0,6.0,2.0
4996,2021Q3,S08000032,Ovarian,30,30,38.0,0.0,661960.0,1.0,6.0,2.0
4997,2021Q3,S08000032,Upper GI,106,106,38.0,0.0,661960.0,1.0,6.0,2.0


In [6]:
## INTERACTIONS 

d=len(df['CancerType'])


# cancer type and region
categorical_columns =['HB','CancerType']


for column in categorical_columns:
     df[column] = pd.factorize(df[column])[0]


N=len(df['CancerType'].unique())
n=len(df['HB'].unique())
Np=sympy.prime(N)
nnp=sympy.prime(n)
x=list(sympy.primerange(0, Np*2))
y=list(sympy.primerange(Np*2,Np*nnp ))

HB_Cancertypes = np.zeros((d,1))


for i in np.arange(d):
    HB_Cancertypes[i]=int(x[df['CancerType'][i]]*y[df['HB'][i]])

df['HB_Cancertypes'] =HB_Cancertypes


# cancer type and date
categorical_columns =['Date','CancerType']


for column in categorical_columns:
     df[column] = pd.factorize(df[column])[0]


N=len(df['CancerType'].unique())
n=len(df['Date'].unique())
Np=sympy.prime(N)
nnp=sympy.prime(n)
x=list(sympy.primerange(0, Np*2))
y=list(sympy.primerange(Np*2,Np*nnp ))

Date_Cancertypes = np.zeros((d,1))


for i in np.arange(d):
    Date_Cancertypes[i]=int(x[df['CancerType'][i]]*y[df['Date'][i]])

df['Date_Cancertypes'] =Date_Cancertypes



# region and date
categorical_columns =['Date','HB']


for column in categorical_columns:
     df[column] = pd.factorize(df[column])[0]


N=len(df['HB'].unique())
n=len(df['Date'].unique())
Np=sympy.prime(N)
nnp=sympy.prime(n)
x=list(sympy.primerange(0, Np*2))
y=list(sympy.primerange(Np*2,Np*nnp ))

Date_HB = np.zeros((d,1))


for i in np.arange(d):
    Date_HB[i]=int(x[df['HB'][i]]*y[df['Date'][i]])

df['Date_HB'] =Date_HB


# cancer type and pandemic
categorical_columns =['Pandemic_cat','CancerType']


for column in categorical_columns:
     df[column] = pd.factorize(df[column])[0]


N=len(df['CancerType'].unique())
n=len(df['Pandemic_cat'].unique())
Np=sympy.prime(N)
nnp=sympy.prime(n)
x=list(sympy.primerange(0, Np*2))
y=list(sympy.primerange(Np*2,Np*nnp ))

Pandemic_cat_Cancertypes = np.zeros((d,1))


for i in np.arange(d):
    Pandemic_cat_Cancertypes[i]=int(x[df['CancerType'][i]]*y[df['Pandemic_cat'][i]])

df['Pandemic_cat_Cancertypes'] =Pandemic_cat_Cancertypes




# cancer type and pandemic
categorical_columns =['Pandemic_cat','HB']


for column in categorical_columns:
     df[column] = pd.factorize(df[column])[0]


N=len(df['HB'].unique())
n=len(df['Pandemic_cat'].unique())
Np=sympy.prime(N)
nnp=sympy.prime(n)
x=list(sympy.primerange(0, Np*2))
y=list(sympy.primerange(Np*2,Np*nnp ))

Pandemic_cat_HB = np.zeros((d,1))


for i in np.arange(d):
    Pandemic_cat_HB[i]=int(x[df['HB'][i]]*y[df['Pandemic_cat'][i]])

df['Pandemic_cat_HB'] =Pandemic_cat_HB



new_columns = ['HB_Cancertypes','Pandemic_cat_HB','Pandemic_cat_Cancertypes','Date_Cancertypes','Date_HB']
for column in new_columns:
     df[column] = pd.factorize(df[column])[0]

df.to_csv('data/31days_interactions.csv')

NameError: name 'sympy' is not defined

In [7]:
# edit 62 days data

df=Data62

# remove Nans
df = df[df['NumberOfEligibleReferrals62DayStandard'].notna()]
df = df[df['NumberOfEligibleReferralsTreatedWithin62Days'].notna()]

# remove 'all cancer types' from data
df = df[df['CancerType'] != 'All Cancer Types']

# add a numerical variable 1-1 to quarters and an index variable before/after pandemic
quarters=df['Quarter']
HBs=np.asarray(df['HB'])
date=np.zeros(len(quarters))
quars=df['Quarter'].unique()
dates=np.arange(len(quars))
pandemic=np.zeros(len(quarters))
pandemic_cat = np.zeros(len(quarters))
justpandemic=np.zeros(len(quarters))
pandemicRecovery=np.zeros(len(quarters))

x=np.where(quars=='2020Q1') # change to decide in which quarter the pandemic begins
z = np.where(quars == '2021Q1') # change to decide where recovery from pandemic begins.
   
pop_size=np.zeros(len(quarters))

for i in np.arange(len(quarters)):
    Q=quarters[i]
    HB=HBs[i]
    pop_size[i]=find_popsize(HB,Q,popsize)
    y=np.where(quars==quarters[i])
    date[i]=dates[y] # change if we don't want the nuemrical variable to be integers (what is best?)
    if dates[y] > x:
        justpandemic[i] = dates[y]-x
        pandemic_cat[i] = 1
    if dates[y] > x and dates[y] < z:
        pandemic[i] = dates[y]-x
    elif dates[y] > z:
        pandemicRecovery[i] = dates[y]-z
        


df['PopSize'] =pop_size.tolist()
df['Date'] =date.tolist()
df['Pandemic'] =pandemic.tolist()
df['Pandemic_cat'] = pandemic_cat.tolist()
df['JustPandemic'] =justpandemic.tolist()
df['PandemicRecovery'] =pandemicRecovery.tolist()
df.to_csv('data/62days.csv')

In [8]:
df

,Quarter,HB,CancerType,NumberOfEligibleReferrals62DayStandard,NumberOfEligibleReferralsTreatedWithin62Days,Date,DateSquared,Pandemic,PopSize,Pandemic_cat,JustPandemic,PandemicRecovery
0,2012Q1,S08000015,Breast,59,57,0.0,0,0.0,373220.0,0.0,0.0,0.0
1,2012Q1,S08000015,Cervical,3,3,0.0,0,0.0,373220.0,0.0,0.0,0.0
2,2012Q1,S08000015,Colorectal,32,30,0.0,0,0.0,373220.0,0.0,0.0,0.0
3,2012Q1,S08000015,Head & Neck,3,3,0.0,0,0.0,373220.0,0.0,0.0,0.0
4,2012Q1,S08000015,Lung,44,43,0.0,0,0.0,373220.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4819,2021Q3,S08000032,Lymphoma,16,16,38.0,1444,0.0,661960.0,1.0,6.0,2.0
4820,2021Q3,S08000032,Melanoma,23,23,38.0,1444,0.0,661960.0,1.0,6.0,2.0
4821,2021Q3,S08000032,Ovarian,8,8,38.0,1444,0.0,661960.0,1.0,6.0,2.0
4822,2021Q3,S08000032,Upper GI,39,35,38.0,1444,0.0,661960.0,1.0,6.0,2.0


In [79]:
## INTERACTIONS 

d=len(df['CancerType'])


# cancer type and region
categorical_columns =['HB','CancerType']


for column in categorical_columns:
     df[column] = pd.factorize(df[column])[0]


N=len(df['CancerType'].unique())
n=len(df['HB'].unique())
Np=sympy.prime(N)
nnp=sympy.prime(n)
x=list(sympy.primerange(0, Np*2))
y=list(sympy.primerange(Np*2,Np*nnp ))

HB_Cancertypes = np.zeros((d,1))


for i in np.arange(d):
    HB_Cancertypes[i]=int(x[df['CancerType'][i]]*y[df['HB'][i]])

df['HB_Cancertypes'] =HB_Cancertypes


# cancer type and date
categorical_columns =['Date','CancerType']


for column in categorical_columns:
     df[column] = pd.factorize(df[column])[0]


N=len(df['CancerType'].unique())
n=len(df['Date'].unique())
Np=sympy.prime(N)
nnp=sympy.prime(n)
x=list(sympy.primerange(0, Np*2))
y=list(sympy.primerange(Np*2,Np*nnp ))

Date_Cancertypes = np.zeros((d,1))


for i in np.arange(d):
    Date_Cancertypes[i]=int(x[df['CancerType'][i]]*y[df['Date'][i]])

df['Date_Cancertypes'] =Date_Cancertypes



# region and date
categorical_columns =['Date','HB']


for column in categorical_columns:
     df[column] = pd.factorize(df[column])[0]


N=len(df['HB'].unique())
n=len(df['Date'].unique())
Np=sympy.prime(N)
nnp=sympy.prime(n)
x=list(sympy.primerange(0, Np*2))
y=list(sympy.primerange(Np*2,Np*nnp ))

Date_HB = np.zeros((d,1))


for i in np.arange(d):
    Date_HB[i]=int(x[df['HB'][i]]*y[df['Date'][i]])

df['Date_HB'] =Date_HB


# cancer type and pandemic
categorical_columns =['Pandemic_cat','CancerType']


for column in categorical_columns:
     df[column] = pd.factorize(df[column])[0]


N=len(df['CancerType'].unique())
n=len(df['Pandemic_cat'].unique())
Np=sympy.prime(N)
nnp=sympy.prime(n)
x=list(sympy.primerange(0, Np*2))
y=list(sympy.primerange(Np*2,Np*nnp ))

Pandemic_cat_Cancertypes = np.zeros((d,1))


for i in np.arange(d):
    Pandemic_cat_Cancertypes[i]=int(x[df['CancerType'][i]]*y[df['Pandemic_cat'][i]])

df['Pandemic_cat_Cancertypes'] =Pandemic_cat_Cancertypes




# cancer type and pandemic
categorical_columns =['Pandemic_cat','HB']


for column in categorical_columns:
     df[column] = pd.factorize(df[column])[0]


N=len(df['HB'].unique())
n=len(df['Pandemic_cat'].unique())
Np=sympy.prime(N)
nnp=sympy.prime(n)
x=list(sympy.primerange(0, Np*2))
y=list(sympy.primerange(Np*2,Np*nnp ))

Pandemic_cat_HB = np.zeros((d,1))


for i in np.arange(d):
    Pandemic_cat_HB[i]=int(x[df['HB'][i]]*y[df['Pandemic_cat'][i]])

df['Pandemic_cat_HB'] =Pandemic_cat_HB



new_columns = ['HB_Cancertypes','Pandemic_cat_HB','Pandemic_cat_Cancertypes','Date_Cancertypes','Date_HB']
for column in new_columns:
     df[column] = pd.factorize(df[column])[0]


In [80]:
df

,Quarter,HB,CancerType,NumberOfEligibleReferrals62DayStandard,NumberOfEligibleReferralsTreatedWithin62Days,Date,DateSquared,Pandemic,PopSize,Pandemic_cat,JustPandemic,PandemicRecovery,HB_Cancertypes,Date_Cancertypes,Date_HB,Pandemic_cat_Cancertypes,Pandemic_cat_HB
0,2012Q1,0,0,59,57,0,0,0.0,373220.0,0,0.0,0.0,0,0,0,0,0
1,2012Q1,0,1,3,3,0,0,0.0,373220.0,0,0.0,0.0,1,1,0,1,0
2,2012Q1,0,2,32,30,0,0,0.0,373220.0,0,0.0,0.0,2,2,0,2,0
3,2012Q1,0,3,3,3,0,0,0.0,373220.0,0,0.0,0.0,3,3,0,3,0
4,2012Q1,0,4,44,43,0,0,0.0,373220.0,0,0.0,0.0,4,4,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4819,2021Q3,13,5,16,16,38,1444,0.0,661960.0,1,6.0,2.0,119,385,545,15,27
4820,2021Q3,13,6,23,23,38,1444,0.0,661960.0,1,6.0,2.0,120,386,545,16,27
4821,2021Q3,13,7,8,8,38,1444,0.0,661960.0,1,6.0,2.0,121,387,545,17,27
4822,2021Q3,13,8,39,35,38,1444,0.0,661960.0,1,6.0,2.0,122,388,545,18,27


In [68]:
df.to_csv('data/62days_interactions.csv')

4824